In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = []
Y = []

d = {}
cnt = 0

for f in os.listdir():
    if f.endswith(".npy"):
        imgs = np.load(f)
        print(imgs.shape,f)
        X.append(imgs)
        labels = np.ones((imgs.shape[0]))*cnt
        Y.append(labels)
        
        d[cnt] = f[:-4]
        cnt +=1

(20, 100, 100, 3) Amit.npy
(20, 100, 100, 3) prateek.npy


In [3]:
X = np.array(X)
Y = np.array(Y)

X = X.reshape((40,100,100,3))
Y = Y.reshape((40,))
print(X.shape,Y.shape)
print(d)

(40, 100, 100, 3) (40,)
{0: 'Amit', 1: 'prateek'}


In [4]:
def dist(p1,p2):
    return np.sum((p1-p2)**2)**.5

def KNN(X,Y,test_point,k=5):
    
    d = []
    m = X.shape[0]
    #1 - Distance of test_point from all points
    for i in range(m):
        current_dist = dist(test_point,X[i])
        d.append((current_dist,Y[i]))
        
    # 2 - Sort 
    d = sorted(d)
    # 3 - Pick the first k points
    d = d[:k]

    # 4 - Create an numpy array 
    d = np.array(d)
    
    d = d[:,1]
    #print(d)
    # 5 - Find out uniq classes and counts
    uniq,cnts = np.unique(d,return_counts=True)
    idx = np.argmax(cnts)
    # 6 -  Find the label with maximum cnt
    pred = uniq[idx]
    # Mapping indices to classname
    return int(pred)
    


In [5]:
detector = cv2.CascadeClassifier('Datasets/FaceCascade/templatedata.xml')
camera = cv2.VideoCapture(0)

In [6]:
while True:
    b,img = camera.read()
    if b==False:
        continue
    # Find out faces--------------------------
    faces = detector.detectMultiScale(img,1.3)
    if(len(faces)==0):
        continue
    
    #Predict Labels for all the faces
    for f in faces:
        f = faces[0]
        x,y,w,h = f
        green = (0,255,0)
        cv2.rectangle(img,(x,y),(x+w,y+h),green,5)
        crop_face = img[y:y+h,x:x+w]
        #-save the cropped face
        small_face = cv2.resize(crop_face,(100,100))
        
        #Make Predictions
        predicted_name = d[KNN(X,Y,small_face)]
        
        cv2.putText(img, predicted_name, (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
        
      
    cv2.imshow("Image ",img)
    key = cv2.waitKey(1)&0xFF
    if key==ord('q'):
        break
    
camera.release()   
cv2.destroyAllWindows()